In [1]:
from typing import List
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain.output_parsers.datetime import DatetimeOutputParser
from langchain.output_parsers.boolean import BooleanOutputParser
from langchain_core.output_parsers import PydanticOutputParser
from langchain_core.exceptions import OutputParserException
from langchain.output_parsers import OutputFixingParser
from dotenv import load_dotenv

In [2]:
load_dotenv()

OPENAI_API_KEY="voc-*"

In [3]:
llm = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0.0,
    api_key=OPENAI_API_KEY
)

## Output Parsers

**String Parser**

In [4]:
llm.invoke("hello")

AIMessage(content='Hello! How can I assist you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 9, 'prompt_tokens': 8, 'total_tokens': 17, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_aa07c96156', 'finish_reason': 'stop', 'logprobs': None}, id='run-a2f1bc1c-ebf2-4da9-a072-8e3fcefd41fc-0', usage_metadata={'input_tokens': 8, 'output_tokens': 9, 'total_tokens': 17, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [5]:
llm.invoke("hello").content

'Hello! How can I assist you today?'

In [6]:
parser = StrOutputParser()

In [7]:
parser.invoke(
    llm.invoke("hello")
)

'Hello! How can I assist you today?'

### Other Parsers

**Datetime**

In [8]:
llm.invoke(
    "Output a random datetime in %Y-%m-%dT%H:%M:%S.%fZ. "
    "Don't say anything else"
)

AIMessage(content='2023-10-05T14:23:45.123456Z', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 16, 'prompt_tokens': 33, 'total_tokens': 49, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_aa07c96156', 'finish_reason': 'stop', 'logprobs': None}, id='run-d4d7272b-f383-4782-92c6-5b6602fc3598-0', usage_metadata={'input_tokens': 33, 'output_tokens': 16, 'total_tokens': 49, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [9]:
parser = DatetimeOutputParser()

In [10]:
parser.invoke(
    llm.invoke(
        "Output a random datetime in %Y-%m-%dT%H:%M:%S.%fZ. "
        "Don't say anything else"
    )
)

datetime.datetime(2023, 10, 5, 14, 23, 45, 123456)

**Boolean**

In [11]:
llm.invoke(
    "Are you an AI? YES or NO only"
)

AIMessage(content='YES', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 1, 'prompt_tokens': 16, 'total_tokens': 17, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_aa07c96156', 'finish_reason': 'stop', 'logprobs': None}, id='run-a5940b15-7ab1-4d29-bc47-08153e5db7f5-0', usage_metadata={'input_tokens': 16, 'output_tokens': 1, 'total_tokens': 17, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [12]:
parser = BooleanOutputParser()

In [13]:
parser.invoke(
    input=llm.invoke(
        "Are you an AI? YES or NO only"
    )
)

True

In [14]:
parser.invoke(
    input=llm.invoke(
        "Are you Human? YES or NO only"
    )
)

False

## Structured

**Dict Schema**

In [15]:
from typing_extensions import Annotated, TypedDict

class UserInfo(TypedDict):
    """User's info."""
    name: Annotated[str, "", "User's name. Defaults to ''"]
    country: Annotated[str, "", "Where the user lives. Defaults to ''"]


In [16]:
llm_with_structure = llm.with_structured_output(UserInfo)

In [17]:
llm_with_structure.invoke(
    "My name is Henrique, and I am from Brazil"
)

{'name': 'Henrique', 'country': 'Brazil'}

In [18]:
llm_with_structure.invoke(
    "The sky is blue"
)

{}

In [19]:
llm_with_structure.invoke(
    "Hello, my name is the same as the capital of the U.S.  "
    "But I'm from a country where we usually associate with kangaroos"
)

{'name': 'Washington', 'country': 'Australia'}

**Pydantic**

In [20]:
from pydantic import BaseModel, Field

class PydanticUserInfo(BaseModel):
    """User's info."""
    name: Annotated[str, Field(description="User's name. Defaults to ''", default=None)]
    country: Annotated[str, Field(description="Where the user lives. Defaults to ''", default=None, )]

In [21]:
llm_with_structure = llm.with_structured_output(PydanticUserInfo)

In [22]:
structured_output = llm_with_structure.invoke("The sky is blue")

In [23]:
structured_output

PydanticUserInfo(name=None, country=None)

In [24]:
print(structured_output.name)

None


In [25]:
print(structured_output.country)

None


In [26]:
structured_output = llm_with_structure.invoke(
    "Hello, my name is the same as the capital of the U.S.  "
    "But I'm from a country where we usually associate with kangaroos"
)

In [27]:
structured_output

PydanticUserInfo(name='Washington', country='Australia')

## Dealing with Errors

In [28]:
class Performer(BaseModel):
    """Filmography info about an actor/actress"""
    name: Annotated[str, Field(description="name of an actor/actress")]
    film_names: Annotated[List[str], Field(description="list of names of films they starred in")]

In [29]:
llm_with_structure = llm.with_structured_output(Performer)

In [30]:
response = llm_with_structure.invoke(
    "Generate the filmography for Scarlett Johansson. Top 5 only"
)
response

Performer(name='Scarlett Johansson', film_names=['Lost in Translation', 'The Avengers', 'Her', 'Marriage Story', 'Black Widow'])

**Fixing Parser**

In [31]:
response.json()

/tmp/ipykernel_128/690762135.py:1: PydanticDeprecatedSince20: The `json` method is deprecated; use `model_dump_json` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  response.json()


'{"name":"Scarlett Johansson","film_names":["Lost in Translation","The Avengers","Her","Marriage Story","Black Widow"]}'

In [32]:
parser = PydanticOutputParser(pydantic_object=Performer)

In [33]:
parser.parse(response.json())

/tmp/ipykernel_128/2130552313.py:1: PydanticDeprecatedSince20: The `json` method is deprecated; use `model_dump_json` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  parser.parse(response.json())


Performer(name='Scarlett Johansson', film_names=['Lost in Translation', 'The Avengers', 'Her', 'Marriage Story', 'Black Widow'])

In [34]:
misformatted_result = "{'name': 'Scarlett Johansson', 'film_names': ['The Avengers']}"

In [35]:
try:
    parser.parse(misformatted_result)
except OutputParserException as e:
    print(e)

Invalid json output: {'name': 'Scarlett Johansson', 'film_names': ['The Avengers']}
For troubleshooting, visit: https://python.langchain.com/docs/troubleshooting/errors/OUTPUT_PARSING_FAILURE 


In [36]:
new_parser = OutputFixingParser.from_llm(parser=parser, llm=llm)

In [37]:
new_parser.parse(misformatted_result)

Performer(name='Scarlett Johansson', film_names=['The Avengers', 'Lost in Translation', 'Marriage Story', 'Black Widow'])